In [ ]:
!pip install statsmodels

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing
from src import config

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [ ]:
def mape(test_data, forecast_data):
    return np.where(test_data != 0,
                    abs(test_data.reset_index(drop=True) - forecast_data.reset_index(drop=True))/test_data.reset_index(drop=True),
                    np.where(forecast_data != 0, 1, 0)).mean()

In [ ]:
def minHWM(df):
    train_size = int(len(df)*0.8)
    df_train, df_test = df[0:train_size], df[train_size:]
    test_index = test.index
    
    df_train_clean = df_train.copy()
    rep_index = df_train_clean[df_train_clean['order_total_price']==0].index
    df_train_clean.loc[rep_index, 'order_total_price'] = 1
    
    cols = df_train.columns
    HWM_table = pd.DataFrame(columns=cols, index=test_index)

    for col in cols:
        HWM1 = ExponentialSmoothing(
            df_train_clean[col],
            seasonal_periods=4,
            trend="add",
            seasonal="add",
            use_boxcox=True,
            initialization_method="estimated",
        ).fit()

        HWM2 = ExponentialSmoothing(
            df_train_clean[col],
            seasonal_periods=4,
            trend="add",
            seasonal="mul",
            use_boxcox=True,
            initialization_method="estimated",
        ).fit()

        HWM3 = ExponentialSmoothing(
            df_train_clean[col],
            seasonal_periods=4,
            trend="add",
            seasonal="add",
            damped_trend=True,
            use_boxcox=True,
            initialization_method="estimated",
        ).fit()

        HWM4 = ExponentialSmoothing(
            df_train_clean[col],
            seasonal_periods=4,
            trend="add",
            seasonal="mul",
            damped_trend=True,
            use_boxcox=True,
            initialization_method="estimated",
        ).fit()

        fcastHWM1 = HWM1.forecast(len(df_test)).rename("Holt's Winters additive trend")
        fcastHWM2 = HWM2.forecast(len(df_test)).rename("Holt's Winters multiplicative trend")
        fcastHWM3 = HWM3.forecast(len(df_test)).rename("Holt's Winters additive damped trend")
        fcastHWM4 = HWM4.forecast(len(df_test)).rename("Holt's Winters multiplicative damped trend")


        # Find lowest MAPE from the 4 HWM models
        min_mape = np.inf
        min_HWM = pd.DataFrame()
        for HWM in [fcastHWM1, fcastHWM2, fcastHWM3, fcastHWM4]:
            cur_mape = mape(df_test[col], HWM)
            if cur_mape < min_mape:
                min_mape = cur_mape
                min_HWM = pd.DataFrame(HWM)
                min_model = HWM.name

        HWM_table[col] = min_HWM
        print('Best Model is: ', min_model)
        print('MAPE: ', min_mape)
        
    return HWM_table

In [ ]:
df = pd.read_parquet(config.INT_FILE_PATH / 'transactions.parquet')

In [ ]:
df.head()

In [ ]:
df2 = df[["order_purchase_timestamp", "order_total_price"]]

In [ ]:
weekly_sales = df.set_index("order_purchase_timestamp").resample("W")[["order_total_price"]].sum()

In [ ]:
# Remove the last few weeks with small values)
weekly_sales_clean = weekly_sales[:-8]

In [ ]:
minHWM(weekly_sales_clean)

## Code below is for Visualization of above MinHWM function

In [ ]:
train_size = int(len(weekly_sales_clean)*0.8)

In [ ]:
train, test = weekly_sales_clean[0:train_size], weekly_sales_clean[train_size:]

In [ ]:
train.columns

In [ ]:
train.shape

In [ ]:
test.shape

## Replace 0 value with small value (1 for this case)

In [ ]:
train[train['order_total_price']==0].index

In [ ]:
train_clean = train.copy()
rep_index = train_clean[train_clean['order_total_price']==0].index
train_clean.loc[rep_index, 'order_total_price'] = 1

## HWM Additive Trend

In [ ]:
HWM1 = ExponentialSmoothing(
    train_clean['order_total_price'],
    seasonal_periods=12,
    trend="add",
    seasonal="add",
    use_boxcox=True,
    initialization_method="estimated",
).fit()

In [ ]:
fcastHWM1 = HWM1.forecast(21).rename("Holt's Winters additive trend")

In [ ]:
plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
plt.title('Order Total Price')
plt.plot(weekly_sales_clean['order_total_price'], marker="o", color="black")
plt.plot(HWM1.fittedvalues, marker="o", color="blue")
(line1,) = plt.plot(fcastHWM1, marker="o", color="blue")
plt.legend([line1],[fcastHWM1.name])

In [ ]:
mape(test['order_total_price'], fcastHWM1)

## HWM Multiplicative Trend

In [ ]:
HWM2 = ExponentialSmoothing(
    train_clean['order_total_price'],
    seasonal_periods=24,
    trend="add",
    seasonal="mul",
    use_boxcox=True,
    initialization_method="estimated",
).fit()

In [ ]:
fcastHWM2 = HWM2.forecast(21).rename("Holt's Winters additive trend")

In [ ]:
plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
plt.title('Order Total Price')
plt.plot(weekly_sales_clean['order_total_price'], marker="o", color="black")
plt.plot(HWM2.fittedvalues, marker="o", color="blue")
(line2,) = plt.plot(fcastHWM2, marker="o", color="blue")
plt.legend([line2],[fcastHWM2.name])

In [ ]:
mape(test['order_total_price'], fcastHWM2)

## HWM Additive with Damped Trend

In [ ]:
HWM3 = ExponentialSmoothing(
    train_clean['order_total_price'],
    seasonal_periods=12,
    trend="add",
    seasonal="add",
    damped_trend=True,
    use_boxcox=True,
    initialization_method="estimated",
).fit()

In [ ]:
fcastHWM3 = HWM3.forecast(50).rename("Holt's Winters additive damped trend")

In [ ]:
plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
plt.title('Order Total Price')
plt.plot(weekly_sales_clean['order_total_price'], marker="o", color="black")
plt.plot(HWM3.fittedvalues, marker="o", color="blue")
(line3,) = plt.plot(fcastHWM3, marker="o", color="blue")
plt.legend([line3],[fcastHWM3.name])

In [ ]:
mape(test['order_total_price'], fcastHWM3)

## HWM Multiplicative with Damped Trend

In [ ]:
HWM4 = ExponentialSmoothing(
    train_clean['order_total_price'],
    seasonal_periods=12,
    trend="add",
    seasonal="mul",
    damped_trend=True,
    use_boxcox=True,
    initialization_method="estimated",
).fit()

In [ ]:
fcastHWM4 = HWM4.forecast(21).rename("Holt's Winters multiplicative damped trend")

In [ ]:
plt.figure(figsize=(12,8))
plt.xticks(rotation=45)
plt.title('Order Total Price')
plt.plot(weekly_sales_clean['order_total_price'], marker="o", color="black")
plt.plot(HWM4.fittedvalues, marker="o", color="blue")
(line4,) = plt.plot(fcastHWM4, marker="o", color="blue")
plt.legend([line4],[fcastHWM4.name])

In [ ]:
mape(test['order_total_price'], fcastHWM4)